In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from datetime import datetime
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# set seed before importing keras to change default shuffle argument to false
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import Flatten
import tensorflow as tf

In [45]:
manoastreamdata = pd.read_csv('daily-discharge-data2011-Mar2020.csv')
manoastreamdata.head()

,Date,Discharge,Base Flow,Rainfall,Temp - High,Temp - Low,Temp - Avg,Temp-Diff,Evapotranspiration,Humidity (%) - Max,Humidity (%) - Avg,Humidity (%) - Min,Wind Speed (mph) - Max,Wind Speed (mph) - Avg,Wind Speed (mph) - Min
0,5/1/2011,6.03,0.00,32.7542,84.9,72,78.45,12.9,77.43476977,81.0,65.7,53.0,15.0,8.2,0.0
1,5/2/2011,39.80,2.95,25.8792,86.0,71.1,78.55,14.9,83.30772434,88.0,73.1,55.0,17.0,7.9,0.0
2,5/3/2011,24.50,4.49,14.4000,84.9,71.1,78,13.8,79.71599484,91.0,71.8,51.0,21.0,11.0,0.0
3,5/4/2011,8.93,4.74,27.3792,84.0,72,78,12,74.33554848,79.0,66.9,53.0,25.0,16.1,9.0
4,5/5/2011,7.70,4.87,31.3167,84.0,73,78.5,11,71.54232467,79.0,64.0,51.0,22.0,15.2,7.0


In [46]:
# Changing string value into pandas datetime object
manoastreamdata.Date = pd.to_datetime(manoastreamdata.Date)
manoastreamdata.head()

,Date,Discharge,Base Flow,Rainfall,Temp - High,Temp - Low,Temp - Avg,Temp-Diff,Evapotranspiration,Humidity (%) - Max,Humidity (%) - Avg,Humidity (%) - Min,Wind Speed (mph) - Max,Wind Speed (mph) - Avg,Wind Speed (mph) - Min
0,2011-05-01,6.03,0.00,32.7542,84.9,72,78.45,12.9,77.43476977,81.0,65.7,53.0,15.0,8.2,0.0
1,2011-05-02,39.80,2.95,25.8792,86.0,71.1,78.55,14.9,83.30772434,88.0,73.1,55.0,17.0,7.9,0.0
2,2011-05-03,24.50,4.49,14.4000,84.9,71.1,78,13.8,79.71599484,91.0,71.8,51.0,21.0,11.0,0.0
3,2011-05-04,8.93,4.74,27.3792,84.0,72,78,12,74.33554848,79.0,66.9,53.0,25.0,16.1,9.0
4,2011-05-05,7.70,4.87,31.3167,84.0,73,78.5,11,71.54232467,79.0,64.0,51.0,22.0,15.2,7.0


In [47]:
dataset = manoastreamdata.set_index("Date")
dataset.head()

,Discharge,Base Flow,Rainfall,Temp - High,Temp - Low,Temp - Avg,Temp-Diff,Evapotranspiration,Humidity (%) - Max,Humidity (%) - Avg,Humidity (%) - Min,Wind Speed (mph) - Max,Wind Speed (mph) - Avg,Wind Speed (mph) - Min
Date,,,,,,,,,,,,,,
2011-05-01,6.03,0.00,32.7542,84.9,72,78.45,12.9,77.43476977,81.0,65.7,53.0,15.0,8.2,0.0
2011-05-02,39.80,2.95,25.8792,86.0,71.1,78.55,14.9,83.30772434,88.0,73.1,55.0,17.0,7.9,0.0
2011-05-03,24.50,4.49,14.4000,84.9,71.1,78,13.8,79.71599484,91.0,71.8,51.0,21.0,11.0,0.0
2011-05-04,8.93,4.74,27.3792,84.0,72,78,12,74.33554848,79.0,66.9,53.0,25.0,16.1,9.0
2011-05-05,7.70,4.87,31.3167,84.0,73,78.5,11,71.54232467,79.0,64.0,51.0,22.0,15.2,7.0


In [48]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3502 entries, 2011-05-01 to NaT
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Discharge               3256 non-null   float64
 1   Base Flow               3256 non-null   float64
 2   Rainfall                3258 non-null   float64
 3   Temp - High             3258 non-null   float64
 4   Temp - Low              3258 non-null   object 
 5   Temp - Avg              3258 non-null   object 
 6   Temp-Diff               3258 non-null   object 
 7   Evapotranspiration      3258 non-null   object 
 8   Humidity (%) - Max      3258 non-null   float64
 9   Humidity (%) - Avg      3258 non-null   float64
 10  Humidity (%) - Min      3258 non-null   float64
 11  Wind Speed (mph) - Max  3258 non-null   float64
 12  Wind Speed (mph) - Avg  3258 non-null   float64
 13  Wind Speed (mph) - Min  3258 non-null   float64
dtypes: float64(10), object(4)
mem

In [49]:
X = dataset.drop(["Discharge"], axis=1)
y = dataset['Discharge']

In [50]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [52]:
X_train = X_train.values.reshape(X_train.shape[0],X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0],X_test.shape[1], 1)
print(X_train.shape)
print(X_test.shape)

(2451, 13, 1)
(1051, 13, 1)


In [56]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [53]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 12, 64)            192       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                19250     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 19,493
Trainable params: 19,493
Non-trainable params: 0
_________________________________________________________________


In [58]:
cnn_history = model.fit(X_train, y_train, epochs=epochs, verbose=2, validation_split=0.3)


Train on 1715 samples, validate on 736 samples
Epoch 1/40


ValueError: could not convert string to float: '-'